# Creating municipality RIDA layers

Assuming that the data we received from Septima by WeTransfer (2 folders with fgb files, plus one separate fgb file) is in the folder `data/RIDA` (from Dropbox [here](https://www.dropbox.com/scl/fo/38gkcvyfex0atdbqe3hfn/h?rlkey=z9xwq2mh6ga03c301b7f66wqx&dl=0))

This script imports each of the fgb layers we need and cuts them into muncipality-sized bites; then filters merges them to create our evaluation layers.

Run from main repo folder!

In [1]:
# import packages
import geopandas as gpd
import pandas as pd
import os
import matplotlib.pyplot as plt

In [3]:
# read in kommuner data and make subfolders
muni = gpd.read_file("./data/municipality-boundaries/municipality-boundaries.gpkg")

In [4]:
# create subfolders for each municipality
for _, row in muni.iterrows():
    os.makedirs(f"data/linestring/{row.kommunekode}", exist_ok=True)
    os.makedirs(f"data/point/{row.kommunekode}", exist_ok=True)
    os.makedirs(f"data/polygon/{row.kommunekode}", exist_ok=True) 

In [5]:
# dictionary of lists: 
# key is type of data (point/linestring/polygon);
# within each item, list of .fgb filenames for that data type

rida_dict = {}

rida_dict["point"] = [
    "facilit_indkoeb.fgb",
    "facilit_overnatning.fgb",
    "facilit_rasteplads.fgb",
    'facilit_service.fgb',  
    'land_besoegs.fgb',  
    "land_fortid.fgb",
    'land_landemaerke.fgb',
    'land_udflugt.fgb',
]

rida_dict["linestring"] = [
    'land_beskyttnatur_linje.fgb',
    'rute_anden.fgb',
    'rute_cykel.fgb',
    'rute_vandre.fgb',
    'vej_basislinje.fgb',
    'vej_hastighedsgraense.fgb',
    'vej_myndighed.fgb',
    'vej_type.fgb'
]

rida_dict["polygon"] = [
    'land_anvendelse.fgb',
    'land_beskyttnatur_flade.fgb',
    'land_ejerskab.fgb',
    'land_frednatpark.fgb',
    'land_vaerdifuld.fgb'
]

geomtype_dict = {
    "point": "Point",
    "linestring": "LineString",
    "polygon": "Polygon"
}

In [7]:
%%time
# read in fgb file; 
# for each of the kommuner, clip fgb file to its extent; 
# then save in corresponding folder
for k, v in rida_dict.items():

    print(f"reading {k} data")
    for fgb in v:
        # read in fgb file
        print(f"\t {fgb}...")
        name = fgb.replace(".fgb", "")

        gdf = gpd.read_file(f"./data/RIDA/{fgb}")       
        
        assert gdf.crs == muni.crs

        # for each municipality: clip, clean, and save
        for _, row in muni.iterrows():
            
            gdf_curr = gdf.copy()
            
            # clip to municipality boundary
            gdf_curr = gdf_curr.clip(row.geometry)
                
            # get the 3 columns "gruppe", "type", "geometry":
            if "gruppe" in gdf_curr.columns:
                gdf_curr = gdf_curr[["gruppe", "type", "geometry"]]
            else:
                gdf_curr = gdf_curr[["type", "geometry"]]
                gdf_curr["gruppe"] = name
                gdf_curr = gdf_curr[["gruppe", "type", "geometry"]]

            # explode
            gdf_curr = gdf_curr.explode(index_parts=False)

            # remove non-expected type geometries
            gdf_curr = gdf_curr[gdf_curr.geometry.type == geomtype_dict[k]].copy()
            gdf_curr = gdf_curr.reset_index(drop=True)

            # save
            if not gdf_curr.empty:
                gdf_curr.to_file(f"./data/{k}/{row.kommunekode}/{name}.gpkg")    
        
        del gdf

reading point data
	 facilit_indkoeb.fgb...
	 facilit_overnatning.fgb...
	 facilit_rasteplads.fgb...
	 facilit_service.fgb...
	 land_besoegs.fgb...
	 land_fortid.fgb...
	 land_landemaerke.fgb...
	 land_udflugt.fgb...
reading linestring data
	 land_beskyttnatur_linje.fgb...
	 rute_anden.fgb...
	 rute_cykel.fgb...
	 rute_vandre.fgb...
	 vej_basislinje.fgb...
	 vej_hastighedsgraense.fgb...
	 vej_myndighed.fgb...
	 vej_type.fgb...
reading polygon data
	 land_anvendelse.fgb...
	 land_beskyttnatur_flade.fgb...
	 land_ejerskab.fgb...
	 land_frednatpark.fgb...
	 land_vaerdifuld.fgb...
CPU times: user 5h 49min 26s, sys: 7min 14s, total: 5h 56min 40s
Wall time: 8h 15min 39s


***
***
***

In [ ]:
def merge_gdfs(gdf_list):
    # make sure all gdfs are the same crs
    assert len(set([gdf.crs for gdf in gdf_list])) == 1
    # make sure we have the same geometries everywhere
    assert len(set([t for gdf in gdf_list for t in gdf.type])) == 1
    # concatenate with pandas
    gdf_main = pd.concat([gdf[["gruppe", "type", "geometry"]].copy() for gdf in gdf_list])
    gdf_main = gdf_main.reset_index(drop=True)
    return gdf_main

In [ ]:
munis_remove = [
    "0161", "0240", "0163", "0411", # no besoeg
    "0269", "0159", # no overnatning
    "0183", "0147", "0190", "0187", "0825", # no fortidsminder
    "0157", "0175" # no udflugt
]

munis = list(muni.kommunekode)
for m in munis_remove:
    munis.remove(m)

In [ ]:
gdf = gpd.read_file("./data/RIDA/")

In [ ]:
# evaluation layers:
# read in all fgb-to-gpkg layers;
# drop not needed columns;
# explode;
# drop not needed geometries;
# filter and merge;
# save in corresponding kommune subfolder
 
for kode in munis:
    
    print(f"start kode {kode}")

    anve = gpd.read_file(f"data/raw/municipality_data/{kode}/anvendelse.gpkg")
    anve = anve[["gruppe", "type", "geometry"]]
    anve = anve.explode(index_parts=False)
    anve = anve[anve.geometry.type == "Polygon"].copy().reset_index(drop=True)

    besk = gpd.read_file(f"data/raw/municipality_data/{kode}/beskyttet_natur.gpkg")
    besk = besk[["type", "geometry"]]
    besk["gruppe"] = "beskyttet_natur"
    besk = besk[["gruppe", "type", "geometry"]]
    besk = besk.explode(index_parts=False)
    besk = besk[besk.geometry.type == "Polygon"].copy().reset_index(drop=True)

    beso = gpd.read_file(f"data/raw/municipality_data/{kode}/besoeg.gpkg")
    beso = beso[["gruppe", "type", "geometry"]]
    beso = beso.explode(index_parts=False)
    beso = beso[beso.geometry.type == "Point"].copy().reset_index(drop=True)

    fort = gpd.read_file(f"data/raw/municipality_data/{kode}/fortidsminder.gpkg")
    fort = fort[["gruppe", "type", "geometry"]]
    fort = fort.explode(index_parts=False)
    fort = fort[fort.geometry.type == "Point"].copy().reset_index(drop=True)

    fred = gpd.read_file(f"data/raw/municipality_data/{kode}/frednatpark.gpkg")
    fred = fred[["type", "geometry"]]
    fred["gruppe"] = "frednatpark"
    fred = fred.explode(index_parts=False)
    fred = fred[fred.geometry.type == "Polygon"].copy().reset_index(drop=True)

    indk = gpd.read_file(f"data/raw/municipality_data/{kode}/indkoeb.gpkg")
    indk = indk[["type", "geometry"]]
    indk["gruppe"] = "indkoeb"
    indk = indk.explode(index_parts=False)
    indk = indk[indk.geometry.type == "Point"].copy().reset_index(drop=True)

    info = gpd.read_file(f"data/raw/municipality_data/{kode}/infoservice.gpkg")
    info = info[["gruppe", "type", "geometry"]]
    info = info.explode(index_parts=False)
    info = info[info.geometry.type == "Point"].copy().reset_index(drop=True)

    land = gpd.read_file(f"data/raw/municipality_data/{kode}/landemaerker.gpkg")
    land = land[["type", "geometry"]]
    land["gruppe"] = "landemaerker"
    land = land.explode(index_parts=False)
    land = land[land.geometry.type == "Point"].copy().reset_index(drop=True)

    over = gpd.read_file(f"data/raw/municipality_data/{kode}/overnatning.gpkg")
    over = over[["type", "geometry"]]
    over["gruppe"] = "overnatning"
    over = over.explode(index_parts=False)
    over = over[over.geometry.type == "Point"].copy().reset_index(drop=True)

    rast = gpd.read_file(f"data/raw/municipality_data/{kode}/rasteplads.gpkg")
    rast = rast[["gruppe", "type", "geometry"]]
    rast = rast.explode(index_parts=False)
    rast = rast[rast.geometry.type == "Point"].copy().reset_index(drop=True)

    udfl = gpd.read_file(f"data/raw/municipality_data/{kode}/udflugt.gpkg")
    udfl = udfl[["gruppe", "type", "geometry"]]
    udfl = udfl.explode(index_parts=False)
    udfl = udfl[udfl.geometry.type == "Point"].copy().reset_index(drop=True)

    vaer = gpd.read_file(f"data/raw/municipality_data/{kode}/vaerdifuld.gpkg")
    vaer = vaer[["type", "geometry"]]
    vaer["gruppe"] = "vaerdifuld"
    vaer = vaer.explode(index_parts=False)
    vaer = vaer[vaer.geometry.type == "Polygon"].copy().reset_index(drop=True)

    print("\t data read in")

    nature = merge_gdfs(
        [
            anve[(anve["gruppe"]=="Naturtyper") & -anve["type"].isin(["Råstofområde", "Gartneri"])].copy(),
            anve[anve["gruppe"]=="Skovinddeling"].copy(),
            besk.copy(),
            vaer[vaer["type"]=="Naturbeskyttelsesområde"].copy(),
            fred.copy()
        ]
    )

    if len(nature) > 0:
        nature.to_file(f"data/raw/municipality_data/{kode}/nature.gpkg", index = False)
        
    culture = merge_gdfs(
        [
            anve[anve["type"].isin(['Bykerne'])].copy(),
            vaer[vaer["type"].isin(
                [
                    'Kulturhistorisk bevaringsværdi', 
                    'Værdifuldt kulturmiljø', 
                    'Bevaringsværdig landskab'
                ]
            )].copy()
        ]
    )

    if len(culture) > 0:
        culture.to_file(
            f"data/raw/municipality_data/{kode}/culture.gpkg", 
            index = False)
        
    sommerhus = anve[
        anve["type"].isin(
            ['Sommerhusområde', 'Sommerhusområde skov']
            )
        ].copy().reset_index(drop=True)
    if len(sommerhus) > 0:
        sommerhus.to_file(
            f"data/raw/municipality_data/{kode}/sommerhus.gpkg", 
            index = False)

    agri = anve[
        anve["gruppe"]=="Dyrkede arealer"].copy().reset_index(
            drop=True)

    if len(agri) > 0:
        agri.to_file(
            f"data/raw/municipality_data/{kode}/agriculture.gpkg", 
            index = False)
        
    verify = merge_gdfs(
        [
            anve[(anve["gruppe"]=="Teknisk areal") & (-anve["type"].isin(["Sportsanlæg"]))].copy(),
            anve[(anve["gruppe"]=="Bymæssig anvendelse") & (anve["type"].isin(["Høj bebyggelse", "Erhverv"]))].copy()
        ]
    )

    if len(verify) > 0:
        verify.to_file(
            f"data/raw/municipality_data/{kode}/verify.gpkg", 
            index = False)
        

    pois = merge_gdfs(
        [beso, fort, udfl, land]
    )

    if len(pois) > 0:
        pois.to_file(
            f"data/raw/municipality_data/{kode}/pois.gpkg", 
            index = False)
        
    faci = merge_gdfs(
        [
            info[-info["type"].isin(["Turistkontor"])].copy(),
            rast
        ]
    )

    if len(faci) > 0:
        faci.to_file(
            f"data/raw/municipality_data/{kode}/facilities.gpkg", 
            index = False)
        
    service = merge_gdfs(
        [
            indk, 
            over, 
            info[info["type"].isin(["Turistkontor"])]
        ]
    )

    if len(service) > 0:
        service.to_file(
            f"data/raw/municipality_data/{kode}/service.gpkg", 
            index = False)
        
    print(f"\t {kode} done")

* ~~0269: no over~~
* ~~"0183", "0147", "0190", "0825": no fortidsminder~~
* ~~"0157", "0175": no udflugt~~
* ~~0161, 0163: no besoeg, no fortidsminder~~
* ~~0240: no besoeg, no overnatning~~
* ~~0411: no besoeg, no indkoeb, no vaerdifuld, no bad (!)~~
* ~~0187: no fortidsminder, no overnatning~~
* 0159: no over, no udflugt

In [ ]:
# SPECIAL CASES: NO OVERNATNING

for kode in ["0269"]:
    
    print(f"start kode {kode}")

    anve = gpd.read_file(f"data/raw/municipality_data/{kode}/anvendelse.gpkg")
    anve = anve[["gruppe", "type", "geometry"]]
    anve = anve.explode(index_parts=False)
    anve = anve[anve.geometry.type == "Polygon"].copy().reset_index(drop=True)

    besk = gpd.read_file(f"data/raw/municipality_data/{kode}/beskyttet_natur.gpkg")
    besk = besk[["type", "geometry"]]
    besk["gruppe"] = "beskyttet_natur"
    besk = besk.explode(index_parts=False)
    besk = besk[besk.geometry.type == "Polygon"].copy().reset_index(drop=True)

    beso = gpd.read_file(f"data/raw/municipality_data/{kode}/besoeg.gpkg")
    beso = beso[["gruppe", "type", "geometry"]]
    beso = beso.explode(index_parts=False)
    beso = beso[beso.geometry.type == "Point"].copy().reset_index(drop=True)

    fort = gpd.read_file(f"data/raw/municipality_data/{kode}/fortidsminder.gpkg")
    fort = fort[["gruppe", "type", "geometry"]]
    fort = fort.explode(index_parts=False)
    fort = fort[fort.geometry.type == "Point"].copy().reset_index(drop=True)

    fred = gpd.read_file(f"data/raw/municipality_data/{kode}/frednatpark.gpkg")
    fred = fred[["type", "geometry"]]
    fred["gruppe"] = "frednatpark"
    fred = fred.explode(index_parts=False)
    fred = fred[fred.geometry.type == "Polygon"].copy().reset_index(drop=True)

    indk = gpd.read_file(f"data/raw/municipality_data/{kode}/indkoeb.gpkg")
    indk = indk[["type", "geometry"]]
    indk["gruppe"] = "indkoeb"
    indk = indk.explode(index_parts=False)
    indk = indk[indk.geometry.type == "Point"].copy().reset_index(drop=True)

    info = gpd.read_file(f"data/raw/municipality_data/{kode}/infoservice.gpkg")
    info = info[["gruppe", "type", "geometry"]]
    info = info.explode(index_parts=False)
    info = info[info.geometry.type == "Point"].copy().reset_index(drop=True)

    land = gpd.read_file(f"data/raw/municipality_data/{kode}/landemaerker.gpkg")
    land = land[["type", "geometry"]]
    land["gruppe"] = "landemaerker"
    land = land.explode(index_parts=False)
    land = land[land.geometry.type == "Point"].copy().reset_index(drop=True)

    # over = gpd.read_file(f"data/raw/municipality_data/{kode}/overnatning.gpkg")
    # over = over[["type", "geometry"]]
    # over["gruppe"] = "overnatning"
    # over = over.explode(index_parts=False)
    # over = over[over.geometry.type == "Point"].copy().reset_index(drop=True)

    rast = gpd.read_file(f"data/raw/municipality_data/{kode}/rasteplads.gpkg")
    rast = rast[["gruppe", "type", "geometry"]]
    rast = rast.explode(index_parts=False)
    rast = rast[rast.geometry.type == "Point"].copy().reset_index(drop=True)

    udfl = gpd.read_file(f"data/raw/municipality_data/{kode}/udflugt.gpkg")
    udfl = udfl[["gruppe", "type", "geometry"]]
    udfl = udfl.explode(index_parts=False)
    udfl = udfl[udfl.geometry.type == "Point"].copy().reset_index(drop=True)

    vaer = gpd.read_file(f"data/raw/municipality_data/{kode}/vaerdifuld.gpkg")
    vaer = vaer[["type", "geometry"]]
    vaer["gruppe"] = "vaerdifuld"
    vaer = vaer.explode(index_parts=False)
    vaer = vaer[vaer.geometry.type == "Polygon"].copy().reset_index(drop=True)

    print("\t data read in")

    nature = merge_gdfs(
        [
            anve[(anve["gruppe"]=="Naturtyper") & -anve["type"].isin(["Råstofområde", "Gartneri"])].copy(),
            anve[anve["gruppe"]=="Skovinddeling"].copy(),
            besk.copy(),
            vaer[vaer["type"]=="Naturbeskyttelsesområde"].copy(),
            fred.copy()
        ]
    )

    if len(nature) > 0:
        nature.to_file(f"data/raw/municipality_data/{kode}/nature.gpkg", index = False)
    
        
    culture = merge_gdfs(
        [
            anve[anve["type"].isin(['Bykerne'])].copy(),
            vaer[vaer["type"].isin(
                [
                    'Kulturhistorisk bevaringsværdi', 
                    'Værdifuldt kulturmiljø', 
                    'Bevaringsværdig landskab'
                ]
            )].copy()
        ]
    )

    if len(culture) > 0:
        culture.to_file(
            f"data/raw/municipality_data/{kode}/culture.gpkg", 
            index = False)
        
    sommerhus = anve[
        anve["type"].isin(
            ['Sommerhusområde', 'Sommerhusområde skov']
            )
        ].copy().reset_index(drop=True)
    if len(sommerhus) > 0:
        sommerhus.to_file(
            f"data/raw/municipality_data/{kode}/sommerhus.gpkg", 
            index = False)

    agri = anve[
        anve["gruppe"]=="Dyrkede arealer"].copy().reset_index(
            drop=True)

    if len(agri) > 0:
        agri.to_file(
            f"data/raw/municipality_data/{kode}/agriculture.gpkg", 
            index = False)
        
    verify = merge_gdfs(
        [
            anve[(anve["gruppe"]=="Teknisk areal") & (-anve["type"].isin(["Sportsanlæg"]))].copy(),
            anve[(anve["gruppe"]=="Bymæssig anvendelse") & (anve["type"].isin(["Høj bebyggelse", "Erhverv"]))].copy()
        ]
    )

    if len(verify) > 0:
        verify.to_file(
            f"data/raw/municipality_data/{kode}/verify.gpkg", 
            index = False)
        

    pois = merge_gdfs(
        [beso, fort, udfl, land]
    )

    if len(pois) > 0:
        pois.to_file(
            f"data/raw/municipality_data/{kode}/pois.gpkg", 
            index = False)
        
    faci = merge_gdfs(
        [
            info[-info["type"].isin(["Turistkontor"])].copy(),
            rast,
        ]
    )

    if len(faci) > 0:
        faci.to_file(
            f"data/raw/municipality_data/{kode}/facilities.gpkg", 
            index = False)
        
    service = merge_gdfs(
        [
            indk, 
            #over, 
            info[info["type"].isin(["Turistkontor"])]]
    )

    if len(service) > 0:
        service.to_file(
            f"data/raw/municipality_data/{kode}/service.gpkg", 
            index = False)
        
    print(f"\t {kode} done")

In [ ]:
# SPECIAL CASES: NO FORTIDSMINDER

for kode in ["0183", "0147", "0190", "0825"]:
    
    print(f"start kode {kode}")

    anve = gpd.read_file(f"data/raw/municipality_data/{kode}/anvendelse.gpkg")
    anve = anve[["gruppe", "type", "geometry"]]
    anve = anve.explode(index_parts=False)
    anve = anve[anve.geometry.type == "Polygon"].copy().reset_index(drop=True)

    besk = gpd.read_file(f"data/raw/municipality_data/{kode}/beskyttet_natur.gpkg")
    besk = besk[["type", "geometry"]]
    besk["gruppe"] = "beskyttet_natur"
    besk = besk.explode(index_parts=False)
    besk = besk[besk.geometry.type == "Polygon"].copy().reset_index(drop=True)

    beso = gpd.read_file(f"data/raw/municipality_data/{kode}/besoeg.gpkg")
    beso = beso[["gruppe", "type", "geometry"]]
    beso = beso.explode(index_parts=False)
    beso = beso[beso.geometry.type == "Point"].copy().reset_index(drop=True)

    # fort = gpd.read_file(f"data/raw/municipality_data/{kode}/fortidsminder.gpkg")
    # fort = fort[["gruppe", "type", "geometry"]]
    # fort = fort.explode(index_parts=False)
    # fort = fort[fort.geometry.type == "Point"].copy().reset_index(drop=True)

    fred = gpd.read_file(f"data/raw/municipality_data/{kode}/frednatpark.gpkg")
    fred = fred[["type", "geometry"]]
    fred["gruppe"] = "frednatpark"
    fred = fred.explode(index_parts=False)
    fred = fred[fred.geometry.type == "Polygon"].copy().reset_index(drop=True)

    indk = gpd.read_file(f"data/raw/municipality_data/{kode}/indkoeb.gpkg")
    indk = indk[["type", "geometry"]]
    indk["gruppe"] = "indkoeb"
    indk = indk.explode(index_parts=False)
    indk = indk[indk.geometry.type == "Point"].copy().reset_index(drop=True)

    info = gpd.read_file(f"data/raw/municipality_data/{kode}/infoservice.gpkg")
    info = info[["gruppe", "type", "geometry"]]
    info = info.explode(index_parts=False)
    info = info[info.geometry.type == "Point"].copy().reset_index(drop=True)

    land = gpd.read_file(f"data/raw/municipality_data/{kode}/landemaerker.gpkg")
    land = land[["type", "geometry"]]
    land["gruppe"] = "landemaerker"
    land = land.explode(index_parts=False)
    land = land[land.geometry.type == "Point"].copy().reset_index(drop=True)

    over = gpd.read_file(f"data/raw/municipality_data/{kode}/overnatning.gpkg")
    over = over[["type", "geometry"]]
    over["gruppe"] = "overnatning"
    over = over.explode(index_parts=False)
    over = over[over.geometry.type == "Point"].copy().reset_index(drop=True)

    rast = gpd.read_file(f"data/raw/municipality_data/{kode}/rasteplads.gpkg")
    rast = rast[["gruppe", "type", "geometry"]]
    rast = rast.explode(index_parts=False)
    rast = rast[rast.geometry.type == "Point"].copy().reset_index(drop=True)

    udfl = gpd.read_file(f"data/raw/municipality_data/{kode}/udflugt.gpkg")
    udfl = udfl[["gruppe", "type", "geometry"]]
    udfl = udfl.explode(index_parts=False)
    udfl = udfl[udfl.geometry.type == "Point"].copy().reset_index(drop=True)

    vaer = gpd.read_file(f"data/raw/municipality_data/{kode}/vaerdifuld.gpkg")
    vaer = vaer[["type", "geometry"]]
    vaer["gruppe"] = "vaerdifuld"
    vaer = vaer.explode(index_parts=False)
    vaer = vaer[vaer.geometry.type == "Polygon"].copy().reset_index(drop=True)

    print("\t data read in")

    nature = merge_gdfs(
        [
            anve[(anve["gruppe"]=="Naturtyper") & -anve["type"].isin(["Råstofområde", "Gartneri"])].copy(),
            anve[anve["gruppe"]=="Skovinddeling"].copy(),
            besk.copy(),
            vaer[vaer["type"]=="Naturbeskyttelsesområde"].copy(),
            fred.copy()
        ]
    )

    if len(nature) > 0:
        nature.to_file(f"data/raw/municipality_data/{kode}/nature.gpkg", index = False)
    
        
    culture = merge_gdfs(
        [
            anve[anve["type"].isin(['Bykerne'])].copy(),
            vaer[vaer["type"].isin(
                [
                    'Kulturhistorisk bevaringsværdi', 
                    'Værdifuldt kulturmiljø', 
                    'Bevaringsværdig landskab'
                ]
            )].copy()
        ]
    )

    if len(culture) > 0:
        culture.to_file(
            f"data/raw/municipality_data/{kode}/culture.gpkg", 
            index = False)
        
    sommerhus = anve[
        anve["type"].isin(
            ['Sommerhusområde', 'Sommerhusområde skov']
            )
        ].copy().reset_index(drop=True)
    if len(sommerhus) > 0:
        sommerhus.to_file(
            f"data/raw/municipality_data/{kode}/sommerhus.gpkg", 
            index = False)

    agri = anve[
        anve["gruppe"]=="Dyrkede arealer"].copy().reset_index(
            drop=True)

    if len(agri) > 0:
        agri.to_file(
            f"data/raw/municipality_data/{kode}/agriculture.gpkg", 
            index = False)
        
    verify = merge_gdfs(
        [
            anve[(anve["gruppe"]=="Teknisk areal") & (-anve["type"].isin(["Sportsanlæg"]))].copy(),
            anve[(anve["gruppe"]=="Bymæssig anvendelse") & (anve["type"].isin(["Høj bebyggelse", "Erhverv"]))].copy()
        ]
    )

    if len(verify) > 0:
        verify.to_file(
            f"data/raw/municipality_data/{kode}/verify.gpkg", 
            index = False)
        

    pois = merge_gdfs(
        [
            beso, 
            #fort, 
            udfl, 
            land]
    )

    if len(pois) > 0:
        pois.to_file(
            f"data/raw/municipality_data/{kode}/pois.gpkg", 
            index = False)
        
    faci = merge_gdfs(
        [
            info[-info["type"].isin(["Turistkontor"])].copy(),
            rast
        ]
    )

    if len(faci) > 0:
        faci.to_file(
            f"data/raw/municipality_data/{kode}/facilities.gpkg", 
            index = False)
        
    service = merge_gdfs(
        [indk, over, info[info["type"].isin(["Turistkontor"])]]
    )

    if len(service) > 0:
        service.to_file(
            f"data/raw/municipality_data/{kode}/service.gpkg", 
            index = False)
        
    print(f"\t {kode} done")

In [ ]:
# SPECIAL CASES: NO UDFLUGT

for kode in ["0157", "0175"]:
    
    print(f"start kode {kode}")

    anve = gpd.read_file(f"data/raw/municipality_data/{kode}/anvendelse.gpkg")
    anve = anve[["gruppe", "type", "geometry"]]
    anve = anve.explode(index_parts=False)
    anve = anve[anve.geometry.type == "Polygon"].copy().reset_index(drop=True)

    besk = gpd.read_file(f"data/raw/municipality_data/{kode}/beskyttet_natur.gpkg")
    besk = besk[["type", "geometry"]]
    besk["gruppe"] = "beskyttet_natur"
    besk = besk.explode(index_parts=False)
    besk = besk[besk.geometry.type == "Polygon"].copy().reset_index(drop=True)

    beso = gpd.read_file(f"data/raw/municipality_data/{kode}/besoeg.gpkg")
    beso = beso[["gruppe", "type", "geometry"]]
    beso = beso.explode(index_parts=False)
    beso = beso[beso.geometry.type == "Point"].copy().reset_index(drop=True)

    fort = gpd.read_file(f"data/raw/municipality_data/{kode}/fortidsminder.gpkg")
    fort = fort[["gruppe", "type", "geometry"]]
    fort = fort.explode(index_parts=False)
    fort = fort[fort.geometry.type == "Point"].copy().reset_index(drop=True)

    fred = gpd.read_file(f"data/raw/municipality_data/{kode}/frednatpark.gpkg")
    fred = fred[["type", "geometry"]]
    fred["gruppe"] = "frednatpark"
    fred = fred.explode(index_parts=False)
    fred = fred[fred.geometry.type == "Polygon"].copy().reset_index(drop=True)

    indk = gpd.read_file(f"data/raw/municipality_data/{kode}/indkoeb.gpkg")
    indk = indk[["type", "geometry"]]
    indk["gruppe"] = "indkoeb"
    indk = indk.explode(index_parts=False)
    indk = indk[indk.geometry.type == "Point"].copy().reset_index(drop=True)

    info = gpd.read_file(f"data/raw/municipality_data/{kode}/infoservice.gpkg")
    info = info[["gruppe", "type", "geometry"]]
    info = info.explode(index_parts=False)
    info = info[info.geometry.type == "Point"].copy().reset_index(drop=True)

    land = gpd.read_file(f"data/raw/municipality_data/{kode}/landemaerker.gpkg")
    land = land[["type", "geometry"]]
    land["gruppe"] = "landemaerker"
    land = land.explode(index_parts=False)
    land = land[land.geometry.type == "Point"].copy().reset_index(drop=True)

    over = gpd.read_file(f"data/raw/municipality_data/{kode}/overnatning.gpkg")
    over = over[["type", "geometry"]]
    over["gruppe"] = "overnatning"
    over = over.explode(index_parts=False)
    over = over[over.geometry.type == "Point"].copy().reset_index(drop=True)

    rast = gpd.read_file(f"data/raw/municipality_data/{kode}/rasteplads.gpkg")
    rast = rast[["gruppe", "type", "geometry"]]
    rast = rast.explode(index_parts=False)
    rast = rast[rast.geometry.type == "Point"].copy().reset_index(drop=True)

    # udfl = gpd.read_file(f"data/raw/municipality_data/{kode}/udflugt.gpkg")
    # udfl = udfl[["gruppe", "type", "geometry"]]
    # udfl = udfl.explode(index_parts=False)
    # udfl = udfl[udfl.geometry.type == "Point"].copy().reset_index(drop=True)

    vaer = gpd.read_file(f"data/raw/municipality_data/{kode}/vaerdifuld.gpkg")
    vaer = vaer[["type", "geometry"]]
    vaer["gruppe"] = "vaerdifuld"
    vaer = vaer.explode(index_parts=False)
    vaer = vaer[vaer.geometry.type == "Polygon"].copy().reset_index(drop=True)

    print("\t data read in")

    nature = merge_gdfs(
        [
            anve[(anve["gruppe"]=="Naturtyper") & -anve["type"].isin(["Råstofområde", "Gartneri"])].copy(),
            anve[anve["gruppe"]=="Skovinddeling"].copy(),
            besk.copy(),
            vaer[vaer["type"]=="Naturbeskyttelsesområde"].copy(),
            fred.copy()
        ]
    )

    if len(nature) > 0:
        nature.to_file(f"data/raw/municipality_data/{kode}/nature.gpkg", index = False)
    
        
    culture = merge_gdfs(
        [
            anve[anve["type"].isin(['Bykerne'])].copy(),
            vaer[vaer["type"].isin(
                [
                    'Kulturhistorisk bevaringsværdi', 
                    'Værdifuldt kulturmiljø', 
                    'Bevaringsværdig landskab'
                ]
            )].copy()
        ]
    )

    if len(culture) > 0:
        culture.to_file(
            f"data/raw/municipality_data/{kode}/culture.gpkg", 
            index = False)
        
    sommerhus = anve[
        anve["type"].isin(
            ['Sommerhusområde', 'Sommerhusområde skov']
            )
        ].copy().reset_index(drop=True)
    if len(sommerhus) > 0:
        sommerhus.to_file(
            f"data/raw/municipality_data/{kode}/sommerhus.gpkg", 
            index = False)

    agri = anve[
        anve["gruppe"]=="Dyrkede arealer"].copy().reset_index(
            drop=True)

    if len(agri) > 0:
        agri.to_file(
            f"data/raw/municipality_data/{kode}/agriculture.gpkg", 
            index = False)
        
    verify = merge_gdfs(
        [
            anve[(anve["gruppe"]=="Teknisk areal") & (-anve["type"].isin(["Sportsanlæg"]))].copy(),
            anve[(anve["gruppe"]=="Bymæssig anvendelse") & (anve["type"].isin(["Høj bebyggelse", "Erhverv"]))].copy()
        ]
    )

    if len(verify) > 0:
        verify.to_file(
            f"data/raw/municipality_data/{kode}/verify.gpkg", 
            index = False)
        

    pois = merge_gdfs(
        [
            beso, 
            fort, 
            # udfl, 
            land
        ]
    )

    if len(pois) > 0:
        pois.to_file(
            f"data/raw/municipality_data/{kode}/pois.gpkg", 
            index = False)
        
    faci = merge_gdfs(
        [
            info[-info["type"].isin(["Turistkontor"])].copy(),
            rast,
        ]
    )

    if len(faci) > 0:
        faci.to_file(
            f"data/raw/municipality_data/{kode}/facilities.gpkg", 
            index = False)
        
    service = merge_gdfs(
        [indk, over, info[info["type"].isin(["Turistkontor"])]]
    )

    if len(service) > 0:
        service.to_file(
            f"data/raw/municipality_data/{kode}/service.gpkg", 
            index = False)
        
    print(f"\t {kode} done")

In [ ]:
# SPECIAL CASES: NO BESOEG, NO FORTIDSMINDER

for kode in ["0161", "0163"]:
    
    print(f"start kode {kode}")

    anve = gpd.read_file(f"data/raw/municipality_data/{kode}/anvendelse.gpkg")
    anve = anve[["gruppe", "type", "geometry"]]
    anve = anve.explode(index_parts=False)
    anve = anve[anve.geometry.type == "Polygon"].copy().reset_index(drop=True)

    besk = gpd.read_file(f"data/raw/municipality_data/{kode}/beskyttet_natur.gpkg")
    besk = besk[["type", "geometry"]]
    besk["gruppe"] = "beskyttet_natur"
    besk = besk.explode(index_parts=False)
    besk = besk[besk.geometry.type == "Polygon"].copy().reset_index(drop=True)

    # beso = gpd.read_file(f"data/raw/municipality_data/{kode}/besoeg.gpkg")
    # beso = beso[["gruppe", "type", "geometry"]]
    # beso = beso.explode(index_parts=False)
    # beso = beso[beso.geometry.type == "Point"].copy().reset_index(drop=True)

    # fort = gpd.read_file(f"data/raw/municipality_data/{kode}/fortidsminder.gpkg")
    # fort = fort[["gruppe", "type", "geometry"]]
    # fort = fort.explode(index_parts=False)
    # fort = fort[fort.geometry.type == "Point"].copy().reset_index(drop=True)

    fred = gpd.read_file(f"data/raw/municipality_data/{kode}/frednatpark.gpkg")
    fred = fred[["type", "geometry"]]
    fred["gruppe"] = "frednatpark"
    fred = fred.explode(index_parts=False)
    fred = fred[fred.geometry.type == "Polygon"].copy().reset_index(drop=True)

    indk = gpd.read_file(f"data/raw/municipality_data/{kode}/indkoeb.gpkg")
    indk = indk[["type", "geometry"]]
    indk["gruppe"] = "indkoeb"
    indk = indk.explode(index_parts=False)
    indk = indk[indk.geometry.type == "Point"].copy().reset_index(drop=True)

    info = gpd.read_file(f"data/raw/municipality_data/{kode}/infoservice.gpkg")
    info = info[["gruppe", "type", "geometry"]]
    info = info.explode(index_parts=False)
    info = info[info.geometry.type == "Point"].copy().reset_index(drop=True)

    land = gpd.read_file(f"data/raw/municipality_data/{kode}/landemaerker.gpkg")
    land = land[["type", "geometry"]]
    land["gruppe"] = "landemaerker"
    land = land.explode(index_parts=False)
    land = land[land.geometry.type == "Point"].copy().reset_index(drop=True)

    over = gpd.read_file(f"data/raw/municipality_data/{kode}/overnatning.gpkg")
    over = over[["type", "geometry"]]
    over["gruppe"] = "overnatning"
    over = over.explode(index_parts=False)
    over = over[over.geometry.type == "Point"].copy().reset_index(drop=True)

    rast = gpd.read_file(f"data/raw/municipality_data/{kode}/rasteplads.gpkg")
    rast = rast[["gruppe", "type", "geometry"]]
    rast = rast.explode(index_parts=False)
    rast = rast[rast.geometry.type == "Point"].copy().reset_index(drop=True)

    udfl = gpd.read_file(f"data/raw/municipality_data/{kode}/udflugt.gpkg")
    udfl = udfl[["gruppe", "type", "geometry"]]
    udfl = udfl.explode(index_parts=False)
    udfl = udfl[udfl.geometry.type == "Point"].copy().reset_index(drop=True)

    vaer = gpd.read_file(f"data/raw/municipality_data/{kode}/vaerdifuld.gpkg")
    vaer = vaer[["type", "geometry"]]
    vaer["gruppe"] = "vaerdifuld"
    vaer = vaer.explode(index_parts=False)
    vaer = vaer[vaer.geometry.type == "Polygon"].copy().reset_index(drop=True)

    print("\t data read in")

    nature = merge_gdfs(
        [
            anve[(anve["gruppe"]=="Naturtyper") & -anve["type"].isin(["Råstofområde", "Gartneri"])].copy(),
            anve[anve["gruppe"]=="Skovinddeling"].copy(),
            besk.copy(),
            vaer[vaer["type"]=="Naturbeskyttelsesområde"].copy(),
            fred.copy()
        ]
    )

    if len(nature) > 0:
        nature.to_file(f"data/raw/municipality_data/{kode}/nature.gpkg", index = False)
    
        
    culture = merge_gdfs(
        [
            anve[anve["type"].isin(['Bykerne'])].copy(),
            vaer[vaer["type"].isin(
                [
                    'Kulturhistorisk bevaringsværdi', 
                    'Værdifuldt kulturmiljø', 
                    'Bevaringsværdig landskab'
                ]
            )].copy()
        ]
    )

    if len(culture) > 0:
        culture.to_file(
            f"data/raw/municipality_data/{kode}/culture.gpkg", 
            index = False)
        
    sommerhus = anve[
        anve["type"].isin(
            ['Sommerhusområde', 'Sommerhusområde skov']
            )
        ].copy().reset_index(drop=True)
    if len(sommerhus) > 0:
        sommerhus.to_file(
            f"data/raw/municipality_data/{kode}/sommerhus.gpkg", 
            index = False)

    agri = anve[
        anve["gruppe"]=="Dyrkede arealer"].copy().reset_index(
            drop=True)

    if len(agri) > 0:
        agri.to_file(
            f"data/raw/municipality_data/{kode}/agriculture.gpkg", 
            index = False)
        
    verify = merge_gdfs(
        [
            anve[(anve["gruppe"]=="Teknisk areal") & (-anve["type"].isin(["Sportsanlæg"]))].copy(),
            anve[(anve["gruppe"]=="Bymæssig anvendelse") & (anve["type"].isin(["Høj bebyggelse", "Erhverv"]))].copy()
        ]
    )

    if len(verify) > 0:
        verify.to_file(
            f"data/raw/municipality_data/{kode}/verify.gpkg", 
            index = False)
        

    pois = merge_gdfs(
        [
            # beso, 
            # fort, 
            udfl, 
            land
        ]
    )

    if len(pois) > 0:
        pois.to_file(
            f"data/raw/municipality_data/{kode}/pois.gpkg", 
            index = False)
        
    faci = merge_gdfs(
        [
            info[-info["type"].isin(["Turistkontor"])].copy(),
            rast,
        ]
    )

    if len(faci) > 0:
        faci.to_file(
            f"data/raw/municipality_data/{kode}/facilities.gpkg", 
            index = False)
        
    service = merge_gdfs(
        [indk, over, info[info["type"].isin(["Turistkontor"])]]
    )

    if len(service) > 0:
        service.to_file(
            f"data/raw/municipality_data/{kode}/service.gpkg", 
            index = False)
        
    print(f"\t {kode} done")

In [ ]:
# SPECIAL CASES: 0240: no besoeg, no overnatning

for kode in ["0240"]:
    
    print(f"start kode {kode}")

    anve = gpd.read_file(f"data/raw/municipality_data/{kode}/anvendelse.gpkg")
    anve = anve[["gruppe", "type", "geometry"]]
    anve = anve.explode(index_parts=False)
    anve = anve[anve.geometry.type == "Polygon"].copy().reset_index(drop=True)

    besk = gpd.read_file(f"data/raw/municipality_data/{kode}/beskyttet_natur.gpkg")
    besk = besk[["type", "geometry"]]
    besk["gruppe"] = "beskyttet_natur"
    besk = besk.explode(index_parts=False)
    besk = besk[besk.geometry.type == "Polygon"].copy().reset_index(drop=True)

    # beso = gpd.read_file(f"data/raw/municipality_data/{kode}/besoeg.gpkg")
    # beso = beso[["gruppe", "type", "geometry"]]
    # beso = beso.explode(index_parts=False)
    # beso = beso[beso.geometry.type == "Point"].copy().reset_index(drop=True)

    fort = gpd.read_file(f"data/raw/municipality_data/{kode}/fortidsminder.gpkg")
    fort = fort[["gruppe", "type", "geometry"]]
    fort = fort.explode(index_parts=False)
    fort = fort[fort.geometry.type == "Point"].copy().reset_index(drop=True)

    fred = gpd.read_file(f"data/raw/municipality_data/{kode}/frednatpark.gpkg")
    fred = fred[["type", "geometry"]]
    fred["gruppe"] = "frednatpark"
    fred = fred.explode(index_parts=False)
    fred = fred[fred.geometry.type == "Polygon"].copy().reset_index(drop=True)

    indk = gpd.read_file(f"data/raw/municipality_data/{kode}/indkoeb.gpkg")
    indk = indk[["type", "geometry"]]
    indk["gruppe"] = "indkoeb"
    indk = indk.explode(index_parts=False)
    indk = indk[indk.geometry.type == "Point"].copy().reset_index(drop=True)

    info = gpd.read_file(f"data/raw/municipality_data/{kode}/infoservice.gpkg")
    info = info[["gruppe", "type", "geometry"]]
    info = info.explode(index_parts=False)
    info = info[info.geometry.type == "Point"].copy().reset_index(drop=True)

    land = gpd.read_file(f"data/raw/municipality_data/{kode}/landemaerker.gpkg")
    land = land[["type", "geometry"]]
    land["gruppe"] = "landemaerker"
    land = land.explode(index_parts=False)
    land = land[land.geometry.type == "Point"].copy().reset_index(drop=True)

    # over = gpd.read_file(f"data/raw/municipality_data/{kode}/overnatning.gpkg")
    # over = over[["type", "geometry"]]
    # over["gruppe"] = "overnatning"
    # over = over.explode(index_parts=False)
    # over = over[over.geometry.type == "Point"].copy().reset_index(drop=True)

    rast = gpd.read_file(f"data/raw/municipality_data/{kode}/rasteplads.gpkg")
    rast = rast[["gruppe", "type", "geometry"]]
    rast = rast.explode(index_parts=False)
    rast = rast[rast.geometry.type == "Point"].copy().reset_index(drop=True)

    udfl = gpd.read_file(f"data/raw/municipality_data/{kode}/udflugt.gpkg")
    udfl = udfl[["gruppe", "type", "geometry"]]
    udfl = udfl.explode(index_parts=False)
    udfl = udfl[udfl.geometry.type == "Point"].copy().reset_index(drop=True)

    vaer = gpd.read_file(f"data/raw/municipality_data/{kode}/vaerdifuld.gpkg")
    vaer = vaer[["type", "geometry"]]
    vaer["gruppe"] = "vaerdifuld"
    vaer = vaer.explode(index_parts=False)
    vaer = vaer[vaer.geometry.type == "Polygon"].copy().reset_index(drop=True)

    print("\t data read in")

    nature = merge_gdfs(
        [
            anve[(anve["gruppe"]=="Naturtyper") & -anve["type"].isin(["Råstofområde", "Gartneri"])].copy(),
            anve[anve["gruppe"]=="Skovinddeling"].copy(),
            besk.copy(),
            vaer[vaer["type"]=="Naturbeskyttelsesområde"].copy(),
            fred.copy()
        ]
    )

    if len(nature) > 0:
        nature.to_file(f"data/raw/municipality_data/{kode}/nature.gpkg", index = False)
    
        
    culture = merge_gdfs(
        [
            anve[anve["type"].isin(['Bykerne'])].copy(),
            vaer[vaer["type"].isin(
                [
                    'Kulturhistorisk bevaringsværdi', 
                    'Værdifuldt kulturmiljø', 
                    'Bevaringsværdig landskab'
                ]
            )].copy()
        ]
    )

    if len(culture) > 0:
        culture.to_file(
            f"data/raw/municipality_data/{kode}/culture.gpkg", 
            index = False)
        
    sommerhus = anve[
        anve["type"].isin(
            ['Sommerhusområde', 'Sommerhusområde skov']
            )
        ].copy().reset_index(drop=True)
    if len(sommerhus) > 0:
        sommerhus.to_file(
            f"data/raw/municipality_data/{kode}/sommerhus.gpkg", 
            index = False)

    agri = anve[
        anve["gruppe"]=="Dyrkede arealer"].copy().reset_index(
            drop=True)

    if len(agri) > 0:
        agri.to_file(
            f"data/raw/municipality_data/{kode}/agriculture.gpkg", 
            index = False)
        
    verify = merge_gdfs(
        [
            anve[(anve["gruppe"]=="Teknisk areal") & (-anve["type"].isin(["Sportsanlæg"]))].copy(),
            anve[(anve["gruppe"]=="Bymæssig anvendelse") & (anve["type"].isin(["Høj bebyggelse", "Erhverv"]))].copy()
        ]
    )

    if len(verify) > 0:
        verify.to_file(
            f"data/raw/municipality_data/{kode}/verify.gpkg", 
            index = False)
        

    pois = merge_gdfs(
        [
            # beso, 
            fort, 
            udfl, 
            land
        ]
    )

    if len(pois) > 0:
        pois.to_file(
            f"data/raw/municipality_data/{kode}/pois.gpkg", 
            index = False)
        
    faci = merge_gdfs(
        [
            info[-info["type"].isin(["Turistkontor"])].copy(),
            rast,
        ]
    )

    if len(faci) > 0:
        faci.to_file(
            f"data/raw/municipality_data/{kode}/facilities.gpkg", 
            index = False)
        
    service = merge_gdfs(
        [
            indk, 
            # over, 
            info[info["type"].isin(["Turistkontor"])]]
    )

    if len(service) > 0:
        service.to_file(
            f"data/raw/municipality_data/{kode}/service.gpkg", 
            index = False)
        
    print(f"\t {kode} done")

In [ ]:
# SPECIAL CASES: 0411: no besoeg, no indkoeb, no vaerdifuld

for kode in ["0411"]:
    
    print(f"start kode {kode}")

    anve = gpd.read_file(f"data/raw/municipality_data/{kode}/anvendelse.gpkg")
    anve = anve[["gruppe", "type", "geometry"]]
    anve = anve.explode(index_parts=False)
    anve = anve[anve.geometry.type == "Polygon"].copy().reset_index(drop=True)

    besk = gpd.read_file(f"data/raw/municipality_data/{kode}/beskyttet_natur.gpkg")
    besk = besk[["type", "geometry"]]
    besk["gruppe"] = "beskyttet_natur"
    besk = besk.explode(index_parts=False)
    besk = besk[besk.geometry.type == "Polygon"].copy().reset_index(drop=True)

    # beso = gpd.read_file(f"data/raw/municipality_data/{kode}/besoeg.gpkg")
    # beso = beso[["gruppe", "type", "geometry"]]
    # beso = beso.explode(index_parts=False)
    # beso = beso[beso.geometry.type == "Point"].copy().reset_index(drop=True)

    fort = gpd.read_file(f"data/raw/municipality_data/{kode}/fortidsminder.gpkg")
    fort = fort[["gruppe", "type", "geometry"]]
    fort = fort.explode(index_parts=False)
    fort = fort[fort.geometry.type == "Point"].copy().reset_index(drop=True)

    fred = gpd.read_file(f"data/raw/municipality_data/{kode}/frednatpark.gpkg")
    fred = fred[["type", "geometry"]]
    fred["gruppe"] = "frednatpark"
    fred = fred.explode(index_parts=False)
    fred = fred[fred.geometry.type == "Polygon"].copy().reset_index(drop=True)

    # indk = gpd.read_file(f"data/raw/municipality_data/{kode}/indkoeb.gpkg")
    # indk = indk[["type", "geometry"]]
    # indk["gruppe"] = "indkoeb"
    # indk = indk.explode(index_parts=False)
    # indk = indk[indk.geometry.type == "Point"].copy().reset_index(drop=True)

    info = gpd.read_file(f"data/raw/municipality_data/{kode}/infoservice.gpkg")
    info = info[["gruppe", "type", "geometry"]]
    info = info.explode(index_parts=False)
    info = info[info.geometry.type == "Point"].copy().reset_index(drop=True)

    land = gpd.read_file(f"data/raw/municipality_data/{kode}/landemaerker.gpkg")
    land = land[["type", "geometry"]]
    land["gruppe"] = "landemaerker"
    land = land.explode(index_parts=False)
    land = land[land.geometry.type == "Point"].copy().reset_index(drop=True)

    over = gpd.read_file(f"data/raw/municipality_data/{kode}/overnatning.gpkg")
    over = over[["type", "geometry"]]
    over["gruppe"] = "overnatning"
    over = over.explode(index_parts=False)
    over = over[over.geometry.type == "Point"].copy().reset_index(drop=True)

    rast = gpd.read_file(f"data/raw/municipality_data/{kode}/rasteplads.gpkg")
    rast = rast[["gruppe", "type", "geometry"]]
    rast = rast.explode(index_parts=False)
    rast = rast[rast.geometry.type == "Point"].copy().reset_index(drop=True)

    udfl = gpd.read_file(f"data/raw/municipality_data/{kode}/udflugt.gpkg")
    udfl = udfl[["gruppe", "type", "geometry"]]
    udfl = udfl.explode(index_parts=False)
    udfl = udfl[udfl.geometry.type == "Point"].copy().reset_index(drop=True)

    # vaer = gpd.read_file(f"data/raw/municipality_data/{kode}/vaerdifuld.gpkg")
    # vaer = vaer[["type", "geometry"]]
    # vaer["gruppe"] = "vaerdifuld"
    # vaer = vaer.explode(index_parts=False)
    # vaer = vaer[vaer.geometry.type == "Polygon"].copy().reset_index(drop=True)

    print("\t data read in")

    nature = merge_gdfs(
        [
            anve[(anve["gruppe"]=="Naturtyper") & -anve["type"].isin(["Råstofområde", "Gartneri"])].copy(),
            anve[anve["gruppe"]=="Skovinddeling"].copy(),
            besk.copy(),
            # vaer[vaer["type"]=="Naturbeskyttelsesområde"].copy(),
            fred.copy()
        ]
    )

    if len(nature) > 0:
        nature.to_file(f"data/raw/municipality_data/{kode}/nature.gpkg", index = False)
    
        
    culture = anve[anve["type"].isin(['Bykerne'])].copy().reset_index(drop=True)

    if len(culture) > 0:
        culture.to_file(
            f"data/raw/municipality_data/{kode}/culture.gpkg", 
            index = False)
        
    sommerhus = anve[
        anve["type"].isin(
            ['Sommerhusområde', 'Sommerhusområde skov']
            )
        ].copy().reset_index(drop=True)
    if len(sommerhus) > 0:
        sommerhus.to_file(
            f"data/raw/municipality_data/{kode}/sommerhus.gpkg", 
            index = False)

    agri = anve[
        anve["gruppe"]=="Dyrkede arealer"].copy().reset_index(
            drop=True)

    if len(agri) > 0:
        agri.to_file(
            f"data/raw/municipality_data/{kode}/agriculture.gpkg", 
            index = False)
        
    # verify = merge_gdfs(
    #     [
    #         anve[(anve["gruppe"]=="Teknisk areal") & (-anve["type"].isin(["Sportsanlæg"]))].copy(),
    #         anve[(anve["gruppe"]=="Bymæssig anvendelse") & (anve["type"].isin(["Høj bebyggelse", "Erhverv"]))].copy()
    #     ]
    # )

    # if len(verify) > 0:
    #     verify.to_file(
    #         f"data/raw/municipality_data/{kode}/verify.gpkg", 
    #         index = False)
        

    pois = merge_gdfs(
        [
            # beso, 
            fort, 
            udfl, 
            land
        ]
    )

    if len(pois) > 0:
        pois.to_file(
            f"data/raw/municipality_data/{kode}/pois.gpkg", 
            index = False)
        
    faci = merge_gdfs(
        [
            info[-info["type"].isin(["Turistkontor"])].copy(),
            rast,
        ]
    )

    if len(faci) > 0:
        faci.to_file(
            f"data/raw/municipality_data/{kode}/facilities.gpkg", 
            index = False)
        
    service = merge_gdfs(
        [indk, over, info[info["type"].isin(["Turistkontor"])]]
    )

    if len(service) > 0:
        service.to_file(
            f"data/raw/municipality_data/{kode}/service.gpkg", 
            index = False)
        
    print(f"\t {kode} done")

In [ ]:
# SPECIAL CASES: 0187: no fortidsminder, no overnatning

for kode in ["0187"]:
    
    print(f"start kode {kode}")

    anve = gpd.read_file(f"data/raw/municipality_data/{kode}/anvendelse.gpkg")
    anve = anve[["gruppe", "type", "geometry"]]
    anve = anve.explode(index_parts=False)
    anve = anve[anve.geometry.type == "Polygon"].copy().reset_index(drop=True)

    besk = gpd.read_file(f"data/raw/municipality_data/{kode}/beskyttet_natur.gpkg")
    besk = besk[["type", "geometry"]]
    besk["gruppe"] = "beskyttet_natur"
    besk = besk.explode(index_parts=False)
    besk = besk[besk.geometry.type == "Polygon"].copy().reset_index(drop=True)

    beso = gpd.read_file(f"data/raw/municipality_data/{kode}/besoeg.gpkg")
    beso = beso[["gruppe", "type", "geometry"]]
    beso = beso.explode(index_parts=False)
    beso = beso[beso.geometry.type == "Point"].copy().reset_index(drop=True)

    # fort = gpd.read_file(f"data/raw/municipality_data/{kode}/fortidsminder.gpkg")
    # fort = fort[["gruppe", "type", "geometry"]]
    # fort = fort.explode(index_parts=False)
    # fort = fort[fort.geometry.type == "Point"].copy().reset_index(drop=True)

    fred = gpd.read_file(f"data/raw/municipality_data/{kode}/frednatpark.gpkg")
    fred = fred[["type", "geometry"]]
    fred["gruppe"] = "frednatpark"
    fred = fred.explode(index_parts=False)
    fred = fred[fred.geometry.type == "Polygon"].copy().reset_index(drop=True)

    indk = gpd.read_file(f"data/raw/municipality_data/{kode}/indkoeb.gpkg")
    indk = indk[["type", "geometry"]]
    indk["gruppe"] = "indkoeb"
    indk = indk.explode(index_parts=False)
    indk = indk[indk.geometry.type == "Point"].copy().reset_index(drop=True)

    info = gpd.read_file(f"data/raw/municipality_data/{kode}/infoservice.gpkg")
    info = info[["gruppe", "type", "geometry"]]
    info = info.explode(index_parts=False)
    info = info[info.geometry.type == "Point"].copy().reset_index(drop=True)

    land = gpd.read_file(f"data/raw/municipality_data/{kode}/landemaerker.gpkg")
    land = land[["type", "geometry"]]
    land["gruppe"] = "landemaerker"
    land = land.explode(index_parts=False)
    land = land[land.geometry.type == "Point"].copy().reset_index(drop=True)

    # over = gpd.read_file(f"data/raw/municipality_data/{kode}/overnatning.gpkg")
    # over = over[["type", "geometry"]]
    # over["gruppe"] = "overnatning"
    # over = over.explode(index_parts=False)
    # over = over[over.geometry.type == "Point"].copy().reset_index(drop=True)

    rast = gpd.read_file(f"data/raw/municipality_data/{kode}/rasteplads.gpkg")
    rast = rast[["gruppe", "type", "geometry"]]
    rast = rast.explode(index_parts=False)
    rast = rast[rast.geometry.type == "Point"].copy().reset_index(drop=True)

    udfl = gpd.read_file(f"data/raw/municipality_data/{kode}/udflugt.gpkg")
    udfl = udfl[["gruppe", "type", "geometry"]]
    udfl = udfl.explode(index_parts=False)
    udfl = udfl[udfl.geometry.type == "Point"].copy().reset_index(drop=True)

    vaer = gpd.read_file(f"data/raw/municipality_data/{kode}/vaerdifuld.gpkg")
    vaer = vaer[["type", "geometry"]]
    vaer["gruppe"] = "vaerdifuld"
    vaer = vaer.explode(index_parts=False)
    vaer = vaer[vaer.geometry.type == "Polygon"].copy().reset_index(drop=True)

    print("\t data read in")

    nature = merge_gdfs(
        [
            anve[(anve["gruppe"]=="Naturtyper") & -anve["type"].isin(["Råstofområde", "Gartneri"])].copy(),
            anve[anve["gruppe"]=="Skovinddeling"].copy(),
            besk.copy(),
            vaer[vaer["type"]=="Naturbeskyttelsesområde"].copy(),
            fred.copy()
        ]
    )

    if len(nature) > 0:
        nature.to_file(f"data/raw/municipality_data/{kode}/nature.gpkg", index = False)
    
        
    culture = merge_gdfs(
        [
            anve[anve["type"].isin(['Bykerne'])].copy(),
            vaer[vaer["type"].isin(
                [
                    'Kulturhistorisk bevaringsværdi', 
                    'Værdifuldt kulturmiljø', 
                    'Bevaringsværdig landskab'
                ]
            )].copy()
        ]
    )

    if len(culture) > 0:
        culture.to_file(
            f"data/raw/municipality_data/{kode}/culture.gpkg", 
            index = False)
        
    sommerhus = anve[
        anve["type"].isin(
            ['Sommerhusområde', 'Sommerhusområde skov']
            )
        ].copy().reset_index(drop=True)
    if len(sommerhus) > 0:
        sommerhus.to_file(
            f"data/raw/municipality_data/{kode}/sommerhus.gpkg", 
            index = False)

    agri = anve[
        anve["gruppe"]=="Dyrkede arealer"].copy().reset_index(
            drop=True)

    if len(agri) > 0:
        agri.to_file(
            f"data/raw/municipality_data/{kode}/agriculture.gpkg", 
            index = False)
        
    verify = merge_gdfs(
        [
            anve[(anve["gruppe"]=="Teknisk areal") & (-anve["type"].isin(["Sportsanlæg"]))].copy(),
            anve[(anve["gruppe"]=="Bymæssig anvendelse") & (anve["type"].isin(["Høj bebyggelse", "Erhverv"]))].copy()
        ]
    )

    if len(verify) > 0:
        verify.to_file(
            f"data/raw/municipality_data/{kode}/verify.gpkg", 
            index = False)
        

    pois = merge_gdfs(
        [
            beso, 
            # fort, 
            udfl, 
            land]
    )

    if len(pois) > 0:
        pois.to_file(
            f"data/raw/municipality_data/{kode}/pois.gpkg", 
            index = False)
        
    faci = merge_gdfs(
        [
            info[-info["type"].isin(["Turistkontor"])].copy(),
            rast,
        ]
    )

    if len(faci) > 0:
        faci.to_file(
            f"data/raw/municipality_data/{kode}/facilities.gpkg", 
            index = False)
        
    service = merge_gdfs(
        [
            indk, 
            # over, 
            info[info["type"].isin(["Turistkontor"])]]
    )

    if len(service) > 0:
        service.to_file(
            f"data/raw/municipality_data/{kode}/service.gpkg", 
            index = False)
        
    print(f"\t {kode} done")

In [ ]:
# SPECIAL CASES: 0159: no over, no udflugt

for kode in ["0159"]:
    
    print(f"start kode {kode}")

    anve = gpd.read_file(f"data/raw/municipality_data/{kode}/anvendelse.gpkg")
    anve = anve[["gruppe", "type", "geometry"]]
    anve = anve.explode(index_parts=False)
    anve = anve[anve.geometry.type == "Polygon"].copy().reset_index(drop=True)

    besk = gpd.read_file(f"data/raw/municipality_data/{kode}/beskyttet_natur.gpkg")
    besk = besk[["type", "geometry"]]
    besk["gruppe"] = "beskyttet_natur"
    besk = besk.explode(index_parts=False)
    besk = besk[besk.geometry.type == "Polygon"].copy().reset_index(drop=True)

    beso = gpd.read_file(f"data/raw/municipality_data/{kode}/besoeg.gpkg")
    beso = beso[["gruppe", "type", "geometry"]]
    beso = beso.explode(index_parts=False)
    beso = beso[beso.geometry.type == "Point"].copy().reset_index(drop=True)

    fort = gpd.read_file(f"data/raw/municipality_data/{kode}/fortidsminder.gpkg")
    fort = fort[["gruppe", "type", "geometry"]]
    fort = fort.explode(index_parts=False)
    fort = fort[fort.geometry.type == "Point"].copy().reset_index(drop=True)

    fred = gpd.read_file(f"data/raw/municipality_data/{kode}/frednatpark.gpkg")
    fred = fred[["type", "geometry"]]
    fred["gruppe"] = "frednatpark"
    fred = fred.explode(index_parts=False)
    fred = fred[fred.geometry.type == "Polygon"].copy().reset_index(drop=True)

    indk = gpd.read_file(f"data/raw/municipality_data/{kode}/indkoeb.gpkg")
    indk = indk[["type", "geometry"]]
    indk["gruppe"] = "indkoeb"
    indk = indk.explode(index_parts=False)
    indk = indk[indk.geometry.type == "Point"].copy().reset_index(drop=True)

    info = gpd.read_file(f"data/raw/municipality_data/{kode}/infoservice.gpkg")
    info = info[["gruppe", "type", "geometry"]]
    info = info.explode(index_parts=False)
    info = info[info.geometry.type == "Point"].copy().reset_index(drop=True)

    land = gpd.read_file(f"data/raw/municipality_data/{kode}/landemaerker.gpkg")
    land = land[["type", "geometry"]]
    land["gruppe"] = "landemaerker"
    land = land.explode(index_parts=False)
    land = land[land.geometry.type == "Point"].copy().reset_index(drop=True)

    # over = gpd.read_file(f"data/raw/municipality_data/{kode}/overnatning.gpkg")
    # over = over[["type", "geometry"]]
    # over["gruppe"] = "overnatning"
    # over = over.explode(index_parts=False)
    # over = over[over.geometry.type == "Point"].copy().reset_index(drop=True)

    rast = gpd.read_file(f"data/raw/municipality_data/{kode}/rasteplads.gpkg")
    rast = rast[["gruppe", "type", "geometry"]]
    rast = rast.explode(index_parts=False)
    rast = rast[rast.geometry.type == "Point"].copy().reset_index(drop=True)

    # udfl = gpd.read_file(f"data/raw/municipality_data/{kode}/udflugt.gpkg")
    # udfl = udfl[["gruppe", "type", "geometry"]]
    # udfl = udfl.explode(index_parts=False)
    # udfl = udfl[udfl.geometry.type == "Point"].copy().reset_index(drop=True)

    vaer = gpd.read_file(f"data/raw/municipality_data/{kode}/vaerdifuld.gpkg")
    vaer = vaer[["type", "geometry"]]
    vaer["gruppe"] = "vaerdifuld"
    vaer = vaer.explode(index_parts=False)
    vaer = vaer[vaer.geometry.type == "Polygon"].copy().reset_index(drop=True)

    print("\t data read in")

    nature = merge_gdfs(
        [
            anve[(anve["gruppe"]=="Naturtyper") & -anve["type"].isin(["Råstofområde", "Gartneri"])].copy(),
            anve[anve["gruppe"]=="Skovinddeling"].copy(),
            besk.copy(),
            vaer[vaer["type"]=="Naturbeskyttelsesområde"].copy(),
            fred.copy()
        ]
    )

    if len(nature) > 0:
        nature.to_file(f"data/raw/municipality_data/{kode}/nature.gpkg", index = False)
    
        
    culture = merge_gdfs(
        [
            anve[anve["type"].isin(['Bykerne'])].copy(),
            vaer[vaer["type"].isin(
                [
                    'Kulturhistorisk bevaringsværdi', 
                    'Værdifuldt kulturmiljø', 
                    'Bevaringsværdig landskab'
                ]
            )].copy()
        ]
    )

    if len(culture) > 0:
        culture.to_file(
            f"data/raw/municipality_data/{kode}/culture.gpkg", 
            index = False)
        
    sommerhus = anve[
        anve["type"].isin(
            ['Sommerhusområde', 'Sommerhusområde skov']
            )
        ].copy().reset_index(drop=True)
    if len(sommerhus) > 0:
        sommerhus.to_file(
            f"data/raw/municipality_data/{kode}/sommerhus.gpkg", 
            index = False)

    agri = anve[
        anve["gruppe"]=="Dyrkede arealer"].copy().reset_index(
            drop=True)

    if len(agri) > 0:
        agri.to_file(
            f"data/raw/municipality_data/{kode}/agriculture.gpkg", 
            index = False)
        
    verify = merge_gdfs(
        [
            anve[(anve["gruppe"]=="Teknisk areal") & (-anve["type"].isin(["Sportsanlæg"]))].copy(),
            anve[(anve["gruppe"]=="Bymæssig anvendelse") & (anve["type"].isin(["Høj bebyggelse", "Erhverv"]))].copy()
        ]
    )

    if len(verify) > 0:
        verify.to_file(
            f"data/raw/municipality_data/{kode}/verify.gpkg", 
            index = False)
        

    pois = merge_gdfs(
        [
            beso, 
            fort, 
            # udfl, 
            land
        ]
    )

    if len(pois) > 0:
        pois.to_file(
            f"data/raw/municipality_data/{kode}/pois.gpkg", 
            index = False)
        
    faci = merge_gdfs(
        [
            info[-info["type"].isin(["Turistkontor"])].copy(),
            rast,
        ]
    )

    if len(faci) > 0:
        faci.to_file(
            f"data/raw/municipality_data/{kode}/facilities.gpkg", 
            index = False)
        
    service = merge_gdfs(
        [
            indk, 
            #over, 
            info[info["type"].isin(["Turistkontor"])]]
    )

    if len(service) > 0:
        service.to_file(
            f"data/raw/municipality_data/{kode}/service.gpkg", 
            index = False)
        
    print(f"\t {kode} done")